In [ ]:
# If running in Google Colab, you may need to run this cell to make sure you're using UTF-8 locale to install LangChain
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -qU torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu pypdf langchain_community langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.3 MB/s eta 0:00:00


In [ ]:
import os
import torch
from pathlib import Path

from typing import List
from fastapi import FastAPI

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
pdf_dir = Path('/content/drive/MyDrive/data')

documents = []

for filename in os.listdir(pdf_dir):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_dir, filename)

        # Load the PDF document
        pdf_loader = PyPDFLoader(pdf_path)
        loaded_docs = pdf_loader.load()

        documents.extend(loaded_docs)
        print(f"Loaded {filename} with {len(loaded_docs)} documents")

Mounted at /content/drive
Loaded software-engineering.pdf with 3 documents
Loaded software-engineering-wiki.pdf with 16 documents
Loaded software-engineering-part1.pdf with 2 documents
Loaded software-engineering-part2.pdf with 2 documents
Loaded Software engineering.pdf with 11 documents
Loaded 1.pdf with 11 documents
Loaded admissions-first-year-apply.pdf with 11 documents
Loaded admissionsgraduate-students.pdf with 4 documents
Loaded admissions-graduate-students.pdf with 4 documents
Loaded how-become-software-engineer.pdf with 5 documents


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(documents)
print(f"Split into {len(chunked_docs)} chunks")
# Check if chunked_docs is empty and handle the error
if not chunked_docs:
    print("Error: Chunked documents are empty. Check your document loading and splitting process.")
    # Handle the error, e.g., by exiting or skipping this step
    # exit()  # or continue to the next step in your workflow
else:
    # Create a vector database if chunked_docs is not empty
    db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))

    # Create a retriever to search the vector store
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

Split into 57 chunks


<ipython-input-6-cdbded61ab44>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
# Create a text generation pipeline using the HuggingFace pipeline
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create a prompt template to generate the prompt for the LLM
prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create a chain with the prompt and the LLM
llm_chain = prompt | llm | StrOutputParser()

<ipython-input-8-b92f5ee89c4e>:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
# Create a chain with the retriever and the LLM chain

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

In [ ]:
question = "What is the Software Architecture and Design course about?"

In [ ]:
llm_chain.invoke({"context": "", "question": question})

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n\n\n</s>\n<|user|>\nWhat is the Software Architecture and Design course about?\n</s>\n<|assistant|>\n\n  The Software Architecture and Design course is focused on teaching students the principles and practices of software architecture and design. It covers topics such as understanding business requirements, defining system boundaries, selecting appropriate technologies, designing scalable and maintainable systems, and creating detailed design artifacts. The course also emphasizes the importance of collaboration and communication between stakeholders throughout the development process. Overall, this course aims to equip students with the skills and knowledge necessary to create high-quality software solutions that meet business needs and are easy to maintain and evolve over time.'

In [ ]:
rag_chain.invoke(question)

'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(metadata={\'source\': \'/content/drive/MyDrive/data/Software engineering.pdf\', \'page\': 3}, page_content=\'do with the characteristic of a certain category or domain of projects.[34] \\nDesign  \\n[edit] \\nMain article:  Software design  \\nSoftware design  is the process of making high -level plans for the software. Design is \\nsometimes divided into levels:  \\n• Interface design  plans the interaction between a system and its environment as well \\nas the inner workings of the system.  \\n• Architectural design  plans the major components of a system, including their\'), Document(metadata={\'source\': \'/content/drive/MyDrive/data/Software engineering.pdf\', \'page\': 6}, page_content=\'most universities teach computer science rather than software engineering, there is a \\nshortage of true software engineers.[40] ETS  (École de technologie supérieure) University \\nand  U

In [ ]:
!pip install uvicorn pydantic

In [ ]:
!pip install pyngrok fastapi uvicorn nest_asyncio --upgrade # Upgrade to the latest version of nest_asyncio

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from langchain import PromptTemplate, LLMChain

# Initialize FastAPI app
app = FastAPI()

# Add CORS middleware
# origins = ["http://localhost", "http://localhost:8080", "http://localhost:4200"]  # Update with your Angular app's origin
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Define a Pydantic model for the request body
class Query(BaseModel):
    query_text: str


# Define the API endpoint

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.post("/query/")
async def query(query: Query):
    query_text = query.query_text  # Access query_text from the model

    # Retrieve relevant documents
    docs = retriever.get_relevant_documents(query_text)

    # Define the prompt template
    template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know.

    {context}

    Question: {question}
    """
    prompt = PromptTemplate(
        template=template,
        input_variables=["context", "question"],
    )
    # Create the chain
    chain = LLMChain(llm=llm, prompt=prompt)  # Assuming 'llm' is your language model
    # Invoke the chain and get the response
    response = await chain.arun(context=docs, question=query_text)
    return {"answer": response}

# Start the FastAPI server
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply() # This line is added to patch the event loop for compatibility with Jupyter Notebooks

    # Start the server on a specific port (e.g., 8090)
    server = uvicorn.Server(config=uvicorn.Config(app, host="127.0.0.1", port=8090))

    async def start_server_and_ngrok():
        await server.serve()  # Start the FastAPI server

    # Create an ngrok tunnel
    ngrok.set_auth_token("2nPjpHvs9rYEDdNq9q3OBf3pp8q_5agvmB5JiEvrbrLrgg3Mw")
    public_url = ngrok.connect(8090).public_url
    print(f"Your FastAPI app is now accessible at: {public_url}")

  # Run the server using uvicorn's run function
    # uvicorn.run(app, host="127.0.0.1", port=8090)
    import asyncio
    loop = asyncio.get_event_loop()
    loop.run_until_complete(start_server_and_ngrok())
    # OR
    # asyncio.run(start_server_and_ngrok()) # If you're using Python 3.7+

INFO:     Started server process [527]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8090 (Press CTRL+C to quit)


Your FastAPI app is now accessible at: https://7992-34-105-70-74.ngrok-free.app
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "GET / HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


<ipython-input-15-ba6933de6d8a>:40: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query_text)
<ipython-input-15-ba6933de6d8a>:54: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)  # Assuming 'llm' is your language model
<ipython-input-15-ba6933de6d8a>:56: LangChainDeprecationWarning: The method `Chain.arun` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~ainvoke` instead.
  response = await chain.arun(context=docs, question=query_text)


INFO:     172.56.216.66:0 - "POST /query/ HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "OPTIONS /query/ HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "POST /query/ HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "POST /query/ HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "OPTIONS /query/ HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "POST /query/ HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "POST /query/ HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "OPTIONS /query/ HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "POST /query/ HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "POST /query/ HTTP/1.1" 200 OK
INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "POST /query/ HTTP/1.1" 200 OK


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


INFO:     2607:fb90:e9bb:8ad4:a530:5f6e:26a3:c628:0 - "POST /query/ HTTP/1.1" 200 OK
